In [ ]:
%run "../Notebooks/initialization.ipynb"

## helper functions

In [1]:
def create_title(mytitle, mytuples):
    mytitle = mytitle+"-"
    for i in range(len(mytuples)):
            mytitle = mytitle+str(mytuples[i][0])+"="+str(mytuples[i][1])+"-"
    mytitle = mytitle[:-1]
    return mytitle

## Image visualization functions

In [2]:
def display_one_tooth(images, toothnum = 0, rows = 1, cols=3, print_suptitle = False, mysuptitle = ""):
    fig, axes = plt.subplots(nrows=rows,ncols=cols,squeeze=False,figsize=(30, 30))
    for i in range(rows):
        for j in range(cols):
            axes[i][j].imshow(plt_format(images[j]))
            #axes[i][j].set_xlabel("tooth_{}".format(toothnum))
            #axes[i][j].axis('off')
    
    cols_title = ['{} View'.format(col) for col in ['External', 'Internal', 'Upper']]
    for ax, c in zip(axes[0], cols_title):
        ax.set_title(c, rotation=0, fontsize=40)
        
    rows_title = ["Tooth_{}             ".format(toothnum)]
    for ax, r in zip(axes[:,0], rows_title):
        ax.set_ylabel(r, rotation=0, fontsize=40)
        
    if print_suptitle == True:
        fig.suptitle("{}".format(mysuptitle),fontsize=40)
    
    fig.tight_layout()    
    plt.show()

In [ ]:
def display_multiple_teeth(images, T, rows = 1, cols=3, print_suptitle = False, mysuptitle = ""):
    
    fig, axes = plt.subplots(nrows=rows,ncols=cols,squeeze=False,figsize=(30, 30))
    for i in range(rows):
        for j in range(cols):
            axes[i][j].imshow(plt_format(images[j][i]))
            #axes[i][j].set_xlabel("tooth_{}".format(toothnum))
            #axes[i][j].axis('off')
    
    cols_title = ['{} View'.format(col) for col in ['External', 'Internal', 'Upper']]
    for ax, c in zip(axes[0], cols_title):
        ax.set_title(c, rotation=0, fontsize=40)
        
    rows_title = ["Tooth_{}             ".format(toothnum) for toothnum in list(range(T[0],T[1]))]
    for ax, r in zip(axes[:,0], rows_title):
        ax.set_ylabel(r, rotation=0, fontsize=40)
        
    if print_suptitle == True:
        fig.suptitle("{}".format(mysuptitle),fontsize=40)
    
    fig.tight_layout()    
    plt.show()

## Outlier removal

In [ ]:
def display_inlier_outlier(cloud, ind,
                           
                           mytitle = None, mytuples = None,
                                                
                           params =None, 
                           configuration_file = None, 
                           take_screen_shot = False,
                           rotate = False,
                           onewindow = True

                          ):
    
    
    inlier_cloud = cloud.select_by_index(ind) #https://github.com/intel-isl/Open3D/issues/1860 
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    if mytitle is None:
        mytitle = "Showing outliers (red) and inliers (gray) "
    
    #if mytuples is not None:
        #mytitle = create_title(mytitle, mytuples)

    outlier_cloud.paint_uniform_color([1, 0, 0]) # print outliers in red
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8]) # print inliers in grey
    
    # use draw geometries to plot
    #o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
                                       #width=1000, height=800,
                                       #window_name='%s'%(mytitle))
    
    
    custom_draw_geometry(inlier_cloud+outlier_cloud, 
                         mytitle = mytitle, mytuples = mytuples,
                         params =params, 
                         configuration_file = configuration_file, 
                         take_screen_shot = take_screen_shot,
                         rotate = rotate,
                         onewindow = onewindow
                        )

## View with custom Visualizer 
http://www.open3d.org/docs/release/tutorial/visualization/customized_visualization.html

In [14]:
# build on this to get your viewer
def custom_draw_geometry(pcd, 
                         mytitle = None, mytuples = None,
                         params =None, 
                         configuration_file = None, 
                         take_screen_shot = False,
                         rotate = False,
                         onewindow = False
                        ):
    # The following code achieves the same effect as:
    # o3d.visualization.draw_geometries([pcd])
    
    if isinstance(pcd,list):
        
        #count of point clouds
        l = len(pcd)
        
        if isinstance(mytitle,list):
            mmytitle = mytitle
        else:
            mmytitle = ["custom draw geometry of pcl_"+str(i+1) for i in list(range(l))]
            
        if isinstance(mytuples,list):
            mmytuples = mytuples
        else:
            mmytuples = [None]*l
        
        if onewindow == False: 
            for i in list(range(len(pcd))):
                custom_draw_geometry(pcd[i], 
                                 mytitle = mmytitle[i], mytuples = mmytuples[i],
                                 params =params, 
                                 configuration_file =configuration_file, 
                                 take_screen_shot = take_screen_shot,
                                 rotate = rotate)
        else:
            all_pc = sum([pcd[i] for i in range(l)])
            custom_draw_geometry(all_pc, 
                 mytitle = mmytitle[i], mytuples = mmytuples[i],
                 params =params, 
                 configuration_file =configuration_file, 
                 take_screen_shot = take_screen_shot,
                 rotate = rotate)
            
    
    
    else: 
        
        # 1- initialize the visualizer
        vis = o3d.visualization.Visualizer()

        #2- set up naming system
        if mytitle is None:
            mytitle = "custom_draw_geometry "

        if mytuples is not None:
            mytitle = create_title(mytitle, mytuples)

        # 3- create widow, otherwise the kernel crashes
        vis.create_window(window_name=mytitle, 
                  width=1000, height=800, 
                  left=50, top=50, 
                  visible=True)

        # 4- add the geometry before taking view control
        #pcd.paint_uniform_color([0.3, 0.3, 0]) 
        vis.add_geometry(pcd)

        # 5- take view control after having added the geometry and before vis.run
        ctr = vis.get_view_control()
        if params is not None: 
            #print ("loading parameters: \n ",params)
            parameters = o3d.io.read_pinhole_camera_parameters(params)
            ctr.convert_from_pinhole_camera_parameters(parameters)

        if configuration_file is not None:
            #print ("loading configuration file: \n ",configuration_file)
            vis.get_render_option().load_from_json(str(configuration_file)) 

        if rotate == True:      
            def rotate_view(vis):
                ctr = vis.get_view_control()
                ctr.rotate(5.0, 0.0) #(speed, direction? )
                return False

            o3d.visualization.draw_geometries_with_animation_callback([pcd],
                                                               rotate_view,
                                                               window_name=mytitle, 
                                                               width=1000, height=800, 
                                                               left=50, top=50)
        elif rotate == "interactive":
        
            def change_background_to_black(vis):
                opt = vis.get_render_option()
                opt.background_color = np.asarray([0, 0, 0])
                return False
            
            def change_background_to_white(vis):
                opt = vis.get_render_option()
                opt.background_color = np.asarray([255, 255, 255])
                return False

            def rotate_view(vis):
                ctr = vis.get_view_control()
                ctr.rotate(10.0, 0.0) #(speed, direction? )
                return False

            def load_render_option(vis):
                vis.get_render_option().load_from_json(
                    myconfiguration_file)
                return False
                                                                      
            def plot_axes(vis):
                opt = vis.get_render_option()
                opt.show_coordinate_frame = not opt.show_coordinate_frame                                                    
                return False
                                                                    

            key_to_callback = {}
            key_to_callback[ord("R")] = load_render_option
            key_to_callback[ord("S")] = rotate_view
            key_to_callback[ord("B")] = change_background_to_black
            key_to_callback[ord("W")] = change_background_to_white
            key_to_callback[ord("A")] = plot_axes
            

            o3d.visualization.draw_geometries_with_key_callbacks([pcd],
                                                                 key_to_callback,                                                                                                                             
                                                                 window_name=mytitle, 
                                                                 width=1000, height=800, 
                                                                 left=50, top=50, 
                                                                 )                                                     
        else: 
            
            vis.run()




        #vis.run()

        if take_screen_shot == True:
            #print ("snipping")
            vis.capture_screen_image(mytitle+'.png')

        vis.destroy_window()

In [11]:
import time
def custom_draw_geometry_outliers(cloud, ind, 
                                  mytitle = None, mytuples = None, 
                                  params = None, #camera parameters,json file (P)
                                  take_screen_shot = False,
                                  fov_step  = None, 
                                  configuration_file = None, #object properties ,json file (O)
                                  rotate = False):
    """
    cloud = point cloud
    ind = index of outliers to be marked in red 
    mytitle = general title
    mytuples = zip(featurelabels,feature) if you want to print which parameter are being used
    fov_step = change of field of View
    
    """
    
    #1- initialize the visualizer
    vis = o3d.visualization.Visualizer()
       
   
    #2- set up naming system
    if mytitle is None:
        mytitle = "Showing outliers (red) and inliers (gray) "
    
    if mytuples is not None:
        mytitle = create_title(mytitle, mytuples)
        #for i in range(len(mytuples)):
            #mytitle = mytitle+"-"+str(mytuples[i][0])+"="+str(mytuples[i][1])+"-"
        #mytitle = mytitle[:-1]

    
    #- create window
    vis.create_window(window_name=mytitle, 
                      width=1000, height=800, 
                      left=50, top=50, 
                      visible=True)
    
    #- get inner and outer cloud
    inlier_cloud = cloud.select_by_index(ind) #https://github.com/intel-isl/Open3D/issues/1860 
    outlier_cloud = cloud.select_by_index(ind, invert=True)
    outlier_cloud.paint_uniform_color([1, 0, 0]) # print outliers in red
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8]) # print inliers in grey
    
    # - add geometries
    vis.add_geometry(inlier_cloud)
    vis.add_geometry(outlier_cloud)
    
    
    # - get view control 
    ctr = vis.get_view_control()
    
    # - set view parameters if any  
    if params is not None: 
        # params = "./ScreenCamera_2021-01-10-15-00-31.json" eg, obtainable by pressing P while visualizing the pcl
        print ("loading parameters: \n ",params)
        parameters = o3d.io.read_pinhole_camera_parameters(params)
        ctr.convert_from_pinhole_camera_parameters(parameters)
    
    if fov_step is not None: 
        #ctr = vis.get_view_control()
        print("Field of view (before changing) %.2f" % ctr.get_field_of_view())
        ctr.change_field_of_view(step=fov_step/5)
        print("Field of view (after changing) %.2f" % ctr.get_field_of_view())
    
    if configuration_file is not None:
        print ("loading configuration file: \n ",configuration_file)
        vis.get_render_option().load_from_json(str(configuration_file)) 
        
    #define pcd
    pcd = inlier_cloud + outlier_cloud
        
    if rotate == True:
             
        def rotate_view(vis):        
            
            vis.get_render_option().load_from_json(
                myconfiguration_file)
            parameters = o3d.io.read_pinhole_camera_parameters(myparams)
            
            
            ctr = vis.get_view_control()
            ctr.rotate(5.0, 0.0) #(speed, direction? )
            #ctr.convert_from_pinhole_camera_parameters(parameters)
            
            return False       
        
        o3d.visualization.draw_geometries_with_animation_callback([pcd],
                                                           rotate_view,
                                                           window_name=mytitle, 
                                                           width=1000, height=800, 
                                                           left=50, top=50, 
                                                          )
    elif rotate == "interactive":
        
        def change_background_to_black(vis):
            opt = vis.get_render_option()
            opt.background_color = np.asarray([0, 0, 0])
            return False
        
        def rotate_view(vis):
            ctr = vis.get_view_control()
            ctr.rotate(10.0, 0.0) #(speed, direction? )
            return False
        
        def load_render_option(vis):
            vis.get_render_option().load_from_json(
                myconfiguration_file)
            return False
        
        
        key_to_callback = {}
        key_to_callback[ord("R")] = load_render_option
        key_to_callback[ord("S")] = rotate_view
        key_to_callback[ord("B")] = change_background_to_black
        
        o3d.visualization.draw_geometries_with_key_callbacks([pcd],
                                                             key_to_callback,                                                                                                                             
                                                             window_name=mytitle, 
                                                             width=1000, height=800, 
                                                             left=50, top=50, 
                                                             )
        
        
        
    else: 
        vis.run()
        #time.sleep(1)
        #close
        #vis.destroy_window()
    
    #time.sleep(1)
    #save the image as png
    #vis.capture_screen_image(mytitle+'.png')
    if take_screen_shot == True:
        print (f"snipping {mytitle}.png")
        vis.capture_screen_image(mytitle+'.png')

    #vis.remove_geometry(pcd)
    #close
    #vis.destroy_window()

In [13]:
def custom_draw_geometry_load_option(pcd, configuration_file):
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window(window_name='custom_load_json', 
                      width=1000, height=800, 
                      left=50, top=50)
    vis.add_geometry(pcd)
    vis.get_render_option().load_from_json(str(configuration_file))    
    vis.run()
    #vis.destroy_window()

In [9]:
def custom_draw_geometry_with_custom_fov(pcd, fov_step):
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name='custom_draw_geometry', 
                      width=1000, height=800, 
                      left=50, top=50, 
                      visible=True)
    
    vis.add_geometry(pcd)
    ctr = vis.get_view_control()
    print("Field of view (before changing) %.2f" % ctr.get_field_of_view())
    ctr.change_field_of_view(step=fov_step)
    print("Field of view (after changing) %.2f" % ctr.get_field_of_view())
    vis.run()
    vis.destroy_window()

In [ ]:
def custom_draw_geometry_with_rotation(pcd):

    def rotate_view(vis):
        ctr = vis.get_view_control()
        ctr.rotate(10.0, 0.0)
        return False

    o3d.visualization.draw_geometries_with_animation_callback([pcd],
                                                              rotate_view)

In [ ]:

def visualize_tooth(t):
    geometry_list = [external_ply[t].paint_uniform_color([0.5, 0, 0]),
                     internal_ply[t].paint_uniform_color([0, 0.5, 0]),
                     upper_ply[t].paint_uniform_color([0, 0, 0.5])
                    ]
    o3d.visualization.draw_geometries(geometry_list,
                                      width=1000, height=800,
                                      window_name='3 raw views of tooth %s'%t
                                     )

def draw_registration_result(source, target, transformation, title = ""):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      width=1000, height=800,
                                      window_name='Open3D-'+str(title)
                                     )

In [ ]:
def find_and_delete_planes(original_cluster_cloud, 
                           ddistance_threshold=0.1,
                           rransac_n=3,
                           nnum_iterations=1000,
                           visualize_on = True):
    """
    returns the point cloud cleaned of planes 
    """
    cluster_cloud = copy.deepcopy(original_cluster_cloud)
    continue_statement = "y"
    delete_statement = False
    
    list_explored_planes = []
    plane_to_explore = False
    
    
    while continue_statement == "y":
    
        while plane_to_explore == False:
            plane_model, inliers = cluster_cloud.segment_plane(distance_threshold=ddistance_threshold,
                                                               ransac_n=rransac_n,
                                                               num_iterations=nnum_iterations
                                                              )
            [a, b, c, d] = plane_model
            
            # if this plane has already been checked, continue changing the parameters
            if [a, b, c, d] in list_explored_planes:
                rransac_n +=0.01
                num_iterations +=1
            
            # mark the plane as now explored and continue
            else: 
                list_explored_planes.append([a, b, c, d])
                plane_to_explore = True
                
                
        print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

        inlier_cloud = cluster_cloud.select_by_index(inliers)
        print ("plane detected contains %s points" %len(inlier_cloud.points))
        inlier_cloud.paint_uniform_color([1.0, 0, 0]) # points on the plane
        temp_cluster_cloud = copy.deepcopy(cluster_cloud)
        outlier_cloud = temp_cluster_cloud.select_by_index(inliers, invert=True)

        #o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

        if visualize_on == True:
            custom_draw_geometry(inlier_cloud+outlier_cloud, 
                             mytitle = "biggest_cluster_cloud_and_outliers", mytuples = "",
                             params =myparams, 
                             configuration_file = myconfiguration_file, 
                             take_screen_shot = False,
                             rotate = True)
            
        print('Delete the plane in red (y/n): ')
        delete_statement = input()
        if str(delete_statement) == "y":
            cluster_cloud = outlier_cloud
        
        print('continue finding planes (y/n): ')
        continue_statement = input()
   
    return cluster_cloud

In [ ]:
def pick_points(pcd):
    #print("")
    #print("1) Please pick at least three correspondences using [shift + left click]")
    #print("   Press [shift + right click] to undo point picking")
    #print("2) After picking points, press 'Q' to close the window")
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window("select 3 points with [shift + left click]; close view with [Q]",
                      width=1000, height=800, 
                      left=50, top=50, 
                      visible=True)
    vis.add_geometry(pcd)
    vis.run()  # user picks points
    vis.destroy_window()
    print("")
    return vis.get_picked_points()

In [ ]:
def demo_crop_geometry(pcd):
    print("Demo for manual geometry cropping")
    print(
        "1) Press 'Y' twice to align geometry with negative direction of y-axis"
    )
    print("2) Press 'K' to lock screen and to switch to selection mode")
    print("3) Drag for rectangle selection,")
    print("   or use ctrl + left click for polygon selection")
    print("4) Press 'C' to get a selected geometry and to save it")
    print("5) Press 'F' to switch to freeview mode")
    temp_pcl = copy.deepcopy(pcd)
    o3d.visualization.draw_geometries_with_editing([temp_pcl])
    
    #print('Confirm selection (y/n): ')
        #delete_statement = input()
        #if str(delete_statement) == "y":
         #   cluster_cloud = outlier_cloud